In [1]:
#import os  
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="1" 

import keras
from keras.datasets import cifar10
from keras import backend as K
from keras.layers import Input, Conv2D, Dense, BatchNormalization, Activation
from keras.layers import GlobalAveragePooling2D, MaxPooling2D, add
from keras.models import Model
from keras.layers import SeparableConv2D

from keras import optimizers,regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import he_normal
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint

num_classes        = 10
batch_size         = 64         # 64 or 32 or other
epochs             = 300
iterations         = 782       
weight_decay=1e-4

log_filepath  = './xception_he_wd_slim'

Using TensorFlow backend.


In [2]:
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
    return x_train, x_test

def scheduler(epoch):
    if epoch < 100:
        return 0.01
    if epoch < 200:
        return 0.001
    return 0.0001

# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes)
x_train, x_test = color_preprocessing(x_train, x_test)

In [3]:
# 36 convolutional layers are structured into 14 modules
def entryflow(x,params,top=False,last=False):
    # modules 2-4,13
    # params is (3,)
    # top = true means module 2, don't use relu
    if last:
        stride = (2,2)
    else:
        stride = (1,1)
    residual = Conv2D(params[0], (1, 1), strides=stride,padding='same',
                     kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(x)
    residual = BatchNormalization(momentum=0.9, epsilon=1e-5)(residual)
    if top:
        x = Activation('relu')(x)
    x = SeparableConv2D(params[1], (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(params[2], (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = MaxPooling2D((3, 3), strides=stride,padding='same')(x)
    x = add([x, residual])
    return x
    
def middleflow(x,params):
    # modules 5-12, params is int
    residual = x
    x = Activation('relu')(x)
    x = SeparableConv2D(params, (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(params, (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(params, (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = add([x, residual])
    return x

def exitflow(x,params):
    # modules 14 , params is (2,)
    x = SeparableConv2D(params[0], (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(params[1], (3, 3),padding='same',
                       depthwise_initializer="he_normal",
                       pointwise_initializer="he_normal",
                       depthwise_regularizer=regularizers.l2(weight_decay),
                       pointwise_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)   
    return x

In [4]:
def xception(img_input,shallow=False, classes=10):
    # modules 1
    x = Conv2D(32,(3, 3),strides=(2, 2),padding='same',
              kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(img_input)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3),strides=(1,1),padding='same',
              kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    # module 2
    x = entryflow(x,(128,128,128),top=True)
    # module 3-4
    x = entryflow(x,(256,256,256))
    x = entryflow(x,(728,728,728))
    # module 5-12
    for _ in range(8):
        x = middleflow(x,728)
    # module 13
    x = entryflow(x,(1024,728,1024),last=True)
    # module 14
    x = exitflow(x,(1536,2048))
    # output
    x = GlobalAveragePooling2D()(x)
    x = Dense(classes, activation='softmax')(x)
    return x

In [ ]:
img_input=Input(shape=(32,32,3))
output = xception(img_input)
model=Model(img_input,output)
model.summary()

In [6]:
# set optimizer
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

# set data augmentation
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant',cval=0.)
datagen.fit(x_train)

# start training
model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                    steps_per_epoch=iterations,
                    epochs=epochs,
                    callbacks=cbks,
                    validation_data=(x_test, y_test))
model.save('xception_he_wd_slim.h5')

Epoch 1/300
782/782 [==============================] - 322s 412ms/step - loss: 5.6460 - acc: 0.4680 - val_loss: 5.3964 - val_acc: 0.5563
Epoch 2/300
782/782 [==============================] - 312s 399ms/step - loss: 5.1249 - acc: 0.6190 - val_loss: 4.9695 - val_acc: 0.6538
Epoch 3/300
782/782 [==============================] - 312s 398ms/step - loss: 4.8282 - acc: 0.6864 - val_loss: 4.8095 - val_acc: 0.6810
Epoch 4/300
782/782 [==============================] - 311s 398ms/step - loss: 4.5991 - acc: 0.7283 - val_loss: 4.6109 - val_acc: 0.7115
Epoch 5/300
782/782 [==============================] - 312s 399ms/step - loss: 4.4042 - acc: 0.7577 - val_loss: 4.3459 - val_acc: 0.7615
Epoch 6/300
782/782 [==============================] - 312s 399ms/step - loss: 4.2309 - acc: 0.7819 - val_loss: 4.2580 - val_acc: 0.7669
Epoch 7/300
782/782 [==============================] - 312s 398ms/step - loss: 4.0793 - acc: 0.7956 - val_loss: 4.0715 - val_acc: 0.7910
Epoch 8/300
782/782 [====================

782/782 [==============================] - 309s 396ms/step - loss: 0.3826 - acc: 0.9998 - val_loss: 0.8339 - val_acc: 0.9207
Epoch 120/300
782/782 [==============================] - 308s 394ms/step - loss: 0.3814 - acc: 0.9997 - val_loss: 0.8384 - val_acc: 0.9199
Epoch 121/300
782/782 [==============================] - 309s 395ms/step - loss: 0.3807 - acc: 0.9996 - val_loss: 0.8347 - val_acc: 0.9213
Epoch 122/300
782/782 [==============================] - 309s 395ms/step - loss: 0.3789 - acc: 0.9998 - val_loss: 0.8386 - val_acc: 0.9209
Epoch 123/300
782/782 [==============================] - 309s 395ms/step - loss: 0.3776 - acc: 0.9998 - val_loss: 0.8431 - val_acc: 0.9198
Epoch 124/300
782/782 [==============================] - 309s 395ms/step - loss: 0.3765 - acc: 0.9998 - val_loss: 0.8343 - val_acc: 0.9219
Epoch 125/300
782/782 [==============================] - 309s 395ms/step - loss: 0.3750 - acc: 0.9999 - val_loss: 0.8330 - val_acc: 0.9211
Epoch 126/300
782/782 [==================

782/782 [==============================] - 312s 400ms/step - loss: 0.3175 - acc: 1.0000 - val_loss: 0.7954 - val_acc: 0.9227
Epoch 179/300
782/782 [==============================] - 312s 399ms/step - loss: 0.3166 - acc: 0.9999 - val_loss: 0.7887 - val_acc: 0.9242
Epoch 180/300
782/782 [==============================] - 312s 399ms/step - loss: 0.3157 - acc: 0.9999 - val_loss: 0.7855 - val_acc: 0.9243
Epoch 181/300
782/782 [==============================] - 312s 399ms/step - loss: 0.3145 - acc: 1.0000 - val_loss: 0.7928 - val_acc: 0.9244
Epoch 182/300
782/782 [==============================] - 312s 399ms/step - loss: 0.3136 - acc: 0.9999 - val_loss: 0.7797 - val_acc: 0.9254
Epoch 183/300
782/782 [==============================] - 313s 400ms/step - loss: 0.3125 - acc: 1.0000 - val_loss: 0.7875 - val_acc: 0.9241
Epoch 184/300
782/782 [==============================] - 312s 399ms/step - loss: 0.3116 - acc: 0.9999 - val_loss: 0.7846 - val_acc: 0.9255
Epoch 185/300
782/782 [==================

782/782 [==============================] - 310s 397ms/step - loss: 0.2927 - acc: 1.0000 - val_loss: 0.7674 - val_acc: 0.9259
Epoch 238/300
782/782 [==============================] - 311s 398ms/step - loss: 0.2926 - acc: 0.9999 - val_loss: 0.7732 - val_acc: 0.9259
Epoch 239/300
782/782 [==============================] - 310s 397ms/step - loss: 0.2925 - acc: 1.0000 - val_loss: 0.7654 - val_acc: 0.9261
Epoch 240/300
782/782 [==============================] - 310s 397ms/step - loss: 0.2924 - acc: 1.0000 - val_loss: 0.7668 - val_acc: 0.9251
Epoch 241/300
782/782 [==============================] - 311s 397ms/step - loss: 0.2923 - acc: 1.0000 - val_loss: 0.7666 - val_acc: 0.9259
Epoch 242/300
782/782 [==============================] - 311s 398ms/step - loss: 0.2923 - acc: 0.9999 - val_loss: 0.7667 - val_acc: 0.9260
Epoch 243/300
782/782 [==============================] - 312s 399ms/step - loss: 0.2922 - acc: 1.0000 - val_loss: 0.7710 - val_acc: 0.9266
Epoch 244/300
782/782 [==================

782/782 [==============================] - 313s 400ms/step - loss: 0.2876 - acc: 1.0000 - val_loss: 0.7604 - val_acc: 0.9267
Epoch 297/300
782/782 [==============================] - 313s 400ms/step - loss: 0.2876 - acc: 1.0000 - val_loss: 0.7682 - val_acc: 0.9257
Epoch 298/300
782/782 [==============================] - 312s 400ms/step - loss: 0.2874 - acc: 1.0000 - val_loss: 0.7623 - val_acc: 0.9261
Epoch 299/300
782/782 [==============================] - 312s 399ms/step - loss: 0.2874 - acc: 1.0000 - val_loss: 0.7638 - val_acc: 0.9272
Epoch 300/300
782/782 [==============================] - 312s 399ms/step - loss: 0.2873 - acc: 1.0000 - val_loss: 0.7620 - val_acc: 0.9262
